In [5]:
#models: https://github.com/RaRe-Technologies/gensim-data

# pre-trained doc2vec: https://github.com/jhlau/doc2vec
# https://github.com/jhlau/doc2vec/blob/master/infer_test.py

import gensim
import gensim.downloader
#from gensim.scripts.glove2word2vec import glove2word2vec

In [6]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing
import os
print(os.getcwd())


/home/frablum/PycharmProjects/learning_sets/notebooks


In [12]:
import glob
from gensim.test.utils import datapath, get_tmpfile
from gensim.corpora import WikiCorpus, MmCorpus

corpora = glob.glob("../corpora/*")
wikis = []
for corpus in corpora:
    path_to_wiki_dump = datapath(os.path.join(os.getcwd(),corpus))
    print(f"create word->word_id mapping for {corpus}..")
    wiki = WikiCorpus(path_to_wiki_dump)  # create word->word_id mapping, ~8h on full wiki
    wikis.append(wiki)
    print("done.")


create word->word_id mapping for ../corpora/enwiki-latest-pages-articles1.xml-p1p41242.bz2..
done.
create word->word_id mapping for ../corpora/frwiki-latest-pages-articles1.xml-p1p306134.bz2..
done.
create word->word_id mapping for ../corpora/enwiki-latest-pages-articles2.xml-p41243p151573.bz2..
done.
create word->word_id mapping for ../corpora/eswiki-latest-pages-articles1.xml-p1p159400.bz2..
done.
create word->word_id mapping for ../corpora/fiwiki-latest-pages-articles.xml.bz2..
done.
create word->word_id mapping for ../corpora/dewiki-latest-pages-articles1.xml-p1p297012.bz2..
done.


In [4]:
"""
corpus_path = get_tmpfile("wiki-corpus.mm")
print("corpus_path",corpus_path)
MmCorpus.serialize(corpus_path, wiki)  # another 8h, creates a file in MatrixMarket format and mapping
"""

'\ncorpus_path = get_tmpfile("wiki-corpus.mm")\nprint("corpus_path",corpus_path)\nMmCorpus.serialize(corpus_path, wiki)  # another 8h, creates a file in MatrixMarket format and mapping\n'

In [13]:
# https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-wikipedia.ipynb
class TaggedWikisDocument(object):
    def __init__(self, wikis):
        self.wikis = wikis
        for wiki in self.wikis:
            wiki.metadata = True
    def __iter__(self):
        for wiki in self.wikis:
            for content, (page_id, title) in wiki.get_texts():
                yield TaggedDocument([c for c in content], [title])
print("creating TaggedWikisDocument..")
documents = TaggedWikisDocument(wikis)
print("done.")

creating TaggedWikisDocument..
done.


In [14]:
pre = Doc2Vec(min_count=19)
pre.scan_vocab(documents)

(649185531, 781235)

In [15]:
"""
print(dir(pre))
for num in range(0, 20):
    print('min_count: {}, size of vocab: '.format(num), pre.scale_vocab(min_count=num, dry_run=True)['memory']['vocab']/700)
"""
print("done.")

done.


In [16]:
cores = multiprocessing.cpu_count()
vector_size=20
epochs=10
models = []
print("PV-DBOW..")
models.append(Doc2Vec(dm=0, vector_size=vector_size, epochs=epochs, dbow_words=1, window=8, min_count=19, workers=cores))
print("done. PV-DM w/average..")
models.append(Doc2Vec(dm=1, vector_size=vector_size, epochs=epochs, dm_mean=1, window=8, min_count=19, workers=cores))
print("done.")

PV-DBOW..
done. PV-DM w/average..
done.


In [17]:
models[0].build_vocab(documents)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d20,n5,w8,mc19,s0.001,t6)
Doc2Vec(dm/m,d20,n5,w8,mc19,s0.001,t6)


In [ ]:
for model in models:
    print(model)
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

Doc2Vec(dbow+w,d20,n5,w8,mc19,s0.001,t6)
Doc2Vec(dm/m,d20,n5,w8,mc19,s0.001,t6)


In [ ]:
print("done.")

In [39]:
for sentence in [['last'],['page'],['last','page'],['last','page']]:
    for model in models:
        print(str(model))
        vec = model.infer_vector(sentence)
        print(vec)
        #pprint(model.dv.most_similar(positive=["the"], topn=20))


Doc2Vec(dbow+w,d20,n5,w8,mc19,s0.001,t6)
[-0.03810085 -0.03745104  0.023528    0.11148287 -0.06903757 -0.01482041
 -0.01568955  0.14675127 -0.19061804  0.18683946  0.01116982 -0.03948331
  0.00580188  0.01739955 -0.01418041  0.06465292  0.04332873 -0.00629203
 -0.09812663 -0.13303563]
Doc2Vec(dm/m,d20,n5,w8,mc19,s0.001,t6)
[-0.08005241  0.07104672 -0.15634401  0.13655604  0.01708053  0.07342408
 -0.13822488  0.04416401 -0.12918773 -0.00642562  0.03768267  0.09772336
 -0.07574216 -0.00097237 -0.01009978 -0.14112754  0.09736551  0.04296375
 -0.10699321 -0.08704165]
Doc2Vec(dbow+w,d20,n5,w8,mc19,s0.001,t6)
[ 1.06171668e-02 -1.47923641e-02  3.58497351e-02  1.59701109e-01
  1.96340829e-02 -6.91482872e-02 -3.79089825e-02  8.82333294e-02
 -1.65948629e-01  1.56536803e-01  5.91934100e-02  9.26303491e-03
 -8.17632899e-02  4.88863792e-03 -4.52252245e-03  7.41044581e-02
  1.52352834e-04  2.09927522e-02 -1.69016182e-01 -1.18401855e-01]
Doc2Vec(dm/m,d20,n5,w8,mc19,s0.001,t6)
[-0.09451106 -0.03155069

In [ ]:
#model = gensim.downloader.load("glove-twitter-25")
#model_filename="enwiki_dbow/doc2vec.bin"
#model = gensim.models.Doc2Vec.load(model_filename)

In [ ]:
models[0].save(os.path.join(os.getcwd(),"doc2vec_model_dbow.bin"))
models[1].save(os.path.join(os.getcwd(),"doc2vec_model_dm.bin"))
#print(model)
#print(dir(model))
#model.get_vector("cat")

In [ ]:
print(models[1].predict_output_word(['bird']))